In [4]:
import os
import pandas as pd
import sqlalchemy
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Create DB engine
db_user = os.getenv("PG_USER")
db_pass = os.getenv("PG_PASSWORD")
db_host = os.getenv("PG_HOST")
db_port = "5432"
db_name = os.getenv("PG_DB")

connection_str = f"postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine = sqlalchemy.create_engine(connection_str)


## Descriptive Analysis

**Business Question:**  
What is the average price, rent estimate, and square footage for Zillow listings grouped by number of bedrooms?

In [5]:
sql_query = '''
WITH valid_listings AS (
    SELECT *
    FROM raw.zillow_data
    WHERE price IS NOT NULL AND rent_estimate IS NOT NULL
)

SELECT 
    beds,
    COUNT(*) AS num_listings,
    ROUND(AVG(price)::NUMERIC, 0) AS avg_price,
    ROUND(AVG(rent_estimate)::NUMERIC, 0) AS avg_rent_estimate,
    ROUND(AVG(sqft)::NUMERIC, 0) AS avg_sqft
FROM valid_listings
GROUP BY beds
ORDER BY beds;
'''

In [6]:
pd.set_option('display.max_rows', None)
df = pd.read_sql(sql_query, engine)
df

,beds,num_listings,avg_price,avg_rent_estimate,avg_sqft
0,1,1,579000.0,2856.0,820.0
1,2,9,1033056.0,4686.0,1384.0
2,3,11,1564254.0,6626.0,1961.0
3,4,4,2106250.0,6998.0,2350.0
4,5,6,3519167.0,6569.0,3724.0
5,6,1,34995000.0,4256.0,13000.0
6,7,3,74981667.0,276867.0,12009.0
7,8,1,126000000.0,5499.0,30610.0
8,9,1,120000000.0,6268.0,14941.0
9,12,1,68000000.0,230541.0,30500.0


**Insight:**  
Listings with 3–4 bedrooms offer a favorable balance between average price ($1.5M–$2.1M), rental estimate (~$6.6K–$7K), and square footage (~2,000–2,350 sqft). In contrast, larger homes (6+ beds) show diminishing returns in rent relative to the steep price increases.

**Recommendation:**  
Focus SFR (single-family rental) investments on 3–4 bedroom homes. These listings deliver more efficient use of capital in terms of rent per dollar spent and appeal to typical renter demographics (families, young professionals).

**Prediction:**  
Markets saturated with mid-sized inventory (3–4 beds) are likely to attract higher institutional demand and maintain better rental yield performance over time.

## Diagnostic Analysis

**Business Question:**  
Which listings offer the highest rental yield (annual rent ÷ price), and how do they compare across different price points?

In [7]:
sql_query = '''
WITH valid_listings AS (
    SELECT *,
           (rent_estimate * 12.0) / NULLIF(price, 0) AS rental_yield
    FROM raw.zillow_data
    WHERE price IS NOT NULL AND rent_estimate IS NOT NULL
)

SELECT 
    address,
    ROUND(price::NUMERIC, 0) AS price,
    ROUND(rent_estimate::NUMERIC, 0) AS rent_estimate,
    ROUND((rental_yield * 100)::NUMERIC, 2) AS rental_yield_pct,
    beds,
    sqft
FROM valid_listings
ORDER BY rental_yield DESC
LIMIT 10;
'''

In [8]:
df = pd.read_sql(sql_query, engine)
df

,address,price,rent_estimate,rental_yield_pct,beds,sqft
0,"3201 Benedict Canyon Dr, Beverly Hills, CA 90210",999000.0,8133.0,9.77,2,1950.0
1,"3548 1/2 Multiview Dr, Los Angeles, CA 90068",2695000.0,19204.0,8.55,3,3602.0
2,"1130 S Flower St APT 118, Los Angeles, CA 90015",595000.0,4163.0,8.40,2,1420.0
3,"10201 Mason Ave UNIT 115, Chatsworth, CA 91311",699999.0,3960.0,6.79,3,1711.0
4,"6225 Coldwater Canyon Ave UNIT 117, North Holl...",549000.0,3026.0,6.61,2,1201.0
5,"1428 W 92nd St, Los Angeles, CA 90047",699999.0,3600.0,6.17,3,1707.0
6,"11767 Iowa Ave APT 1, Los Angeles, CA 90025",680000.0,3441.0,6.07,2,995.0
7,"9340 Yolanda Ave, Northridge, CA 91324",949000.0,4802.0,6.07,4,1500.0
8,"4211 Arch Dr APT 204, Studio City, CA 91604",579000.0,2856.0,5.92,1,820.0
9,"11767 Iowa Ave APT 6, Los Angeles, CA 90025",685000.0,3373.0,5.91,2,936.0


**Insight:**  
The top 10 listings by rental yield offer between 5.9%–9.8% annual return based on rent, with properties priced under $1M performing especially well. Notably, several high-yield properties are located in less premium neighborhoods, suggesting potential for strong returns outside luxury zones.

**Recommendation:**  
Target investment opportunities under $1M with rental yield above 6% — especially in submarkets like Northridge, Studio City, and Chatsworth. These offer attractive returns without the capital lock-up of luxury assets.

**Prediction:**  
As yield compression continues in premium real estate markets, mid-range SFRs with high rental yield will attract more institutional investment, driving up competition and prices in these neighborhoods.


## Bonus Query: Fulfilling JOIN + Window Function Requirement

**Business Question:**  
Which Zillow listings rank highest in rental yield per bedroom count?

This query joins the main table with a filtered high-yield subset and ranks listings using `ROW_NUMBER()` within each `bedroom` group.


In [9]:
sql_query = '''
WITH valid_listings AS (
    SELECT *,
           (rent_estimate * 12.0) / NULLIF(price, 0) AS rental_yield
    FROM raw.zillow_data
    WHERE price IS NOT NULL AND rent_estimate IS NOT NULL
),

high_yield AS (
    SELECT *
    FROM valid_listings
    WHERE (rent_estimate * 12.0) / NULLIF(price, 0) > 0.05
)

SELECT 
    v.address,
    v.beds,
    v.price,
    v.rent_estimate,
    ROUND((v.rental_yield * 100)::NUMERIC, 2) AS rental_yield_pct,
    ROW_NUMBER() OVER (PARTITION BY v.beds ORDER BY v.rental_yield DESC) AS rank_within_bed_group
FROM valid_listings v
JOIN high_yield h
  ON v.address = h.address
ORDER BY v.beds, rank_within_bed_group
LIMIT 10;
'''


In [10]:
df = pd.read_sql(sql_query, engine)
df

,address,beds,price,rent_estimate,rental_yield_pct,rank_within_bed_group
0,"4211 Arch Dr APT 204, Studio City, CA 91604",1,579000,2856.0,5.92,1
1,"3201 Benedict Canyon Dr, Beverly Hills, CA 90210",2,999000,8133.0,9.77,1
2,"1130 S Flower St APT 118, Los Angeles, CA 90015",2,595000,4163.0,8.40,2
3,"6225 Coldwater Canyon Ave UNIT 117, North Holl...",2,549000,3026.0,6.61,3
4,"11767 Iowa Ave APT 1, Los Angeles, CA 90025",2,680000,3441.0,6.07,4
5,"11767 Iowa Ave APT 6, Los Angeles, CA 90025",2,685000,3373.0,5.91,5
6,"3548 1/2 Multiview Dr, Los Angeles, CA 90068",3,2695000,19204.0,8.55,1
7,"10201 Mason Ave UNIT 115, Chatsworth, CA 91311",3,699999,3960.0,6.79,2
8,"1428 W 92nd St, Los Angeles, CA 90047",3,699999,3600.0,6.17,3
9,"3735 Montuso Pl, Encino, CA 91436",3,1699900,8023.0,5.66,4


**Insight:**  
Among Zillow listings with rental yields above 5%, smaller properties (1–3 beds) dominate the top rankings. For example, the highest-yield 2-bed listing returns 9.8%, while several 3-bed units offer over 8.5% yield.

**Recommendation:**  
When prioritizing yield efficiency within bedroom categories, investors should focus on the top 1–2 ranked properties, especially in mid-size SFR segments (2–3 beds).

**Prediction:**  
As rental yield becomes a more important metric in capital deployment decisions, high-ranking listings within each bedroom segment will command a premium in competitive markets.